# Necessary imports

In [14]:
import numpy as np
from pyquil.quil import Program
from pyquil.gates import *

from pyquil.resource_manager import (DirectQubit)
from pyquil.quilbase import *

# from pyquil.quilbase import (Instr, Addr, DefGate, Gate, Measurement, If, While, InstructionGroup)

# from pyquil.quilbase import (InstructionGroup, Instr,
#                        Slot,
#                        Addr,
#                        While,
#                        If,
#                        DefGate,
#                        Gate)

from qvm import tensor_gates
from unitary_generator import apply_gate, permutation_arbitrary, lifted_gate
from gates import gate_matrix, utility_gates

## Helper functions

In [5]:
def value(param_obj):
    if isinstance(param_obj, (float, int, long)):
        return param_obj
    elif isinstance(param_obj, DirectQubit):
        return param_obj._index
    elif isinstance(param_obj, Addr):
        return param_obj.address
    elif isinstance(param_obj, Slot):
        return param_obj.value()

## Control flow

In [49]:
p = Program().defgate('id', [[1.0, 0.0], [0.0, 1.0]])
branch_a = Program().inst(('id', 0)).inst(H(0), MEASURE(0), WAIT, RESET, HALT, H(0), NOP).inst(TRUE(0))
branch_b = Program().defgate('HAD', np.array([[1, 1], [1, -1]]) / np.sqrt(2.)).inst(('HAD', 2))
p.if_then(0, branch_a, branch_b)

In [50]:
print p
print p.get_qubits()

DEFGATE id:
    1.0, 0.0
    0.0, 1.0

JUMP-WHEN @THEN19 [0]
HAD 2
JUMP @END20
LABEL @THEN19
id 0
H 0
MEASURE 0
WAIT
RESET
HALT
H 0
NOP
TRUE [0]
LABEL @END20

set([])


In [51]:
p_obj = p.synthesize()
p_obj

 <JumpTarget @THEN23>,
 <Gate: H 0>,
 <Gate: H 0>,
 <JumpTarget @END24>]

In [67]:
p_obj[0].target == p_obj[3].label

True

In [33]:
for i, e in enumerate(p.actions):
    print e[1].condition
    print e[1].Then
    print e[1].Else

[0]
id 0
H 0
MEASURE 0
WAIT
RESET
HALT
H 0
NOP
TRUE(0)

HAD 2



In [22]:
print p

DEFGATE id:
    1.0, 0.0
    0.0, 1.0

JUMP-WHEN @THEN19 [0]
HAD 2
JUMP @END20
LABEL @THEN19
id 0
H 0
MEASURE 0
WAIT
RESET
HALT
H 0
NOP
TRUE(0)
LABEL @END20



## Raw control (Jump, Label)

In [6]:
p = Program().defgate('id', [[1.0, 0.0], [0.0, 1.0]])
p.inst(('LABEL', 'gcd-one'), H(3), TRUE(0), ('JUMP', 'gcd-one'), ('id', 3))
print p

DEFGATE id:
    1.0, 0.0
    0.0, 1.0

LABEL gcd-one
H 3
TRUE [0]
JUMP gcd-one
id 3



In [11]:
print p

DEFGATE id:
    1.0, 0.0
    0.0, 1.0

LABEL gcd-one
H 3
TRUE [0]
JUMP gcd-one
id 3



In [12]:
p_obj = p.synthesize()
p_obj

 <Gate: H 3>,

In [13]:
for i in p_obj:
    if isinstance(i, Instr):
        print i.operator_name
    print i.

LABEL
H
JUMP
id


In [34]:
for i, action in enumerate(p.actions):
    print isinstance(action[1], Instr)
    print action[1]

True
<Instr LABEL>
True
H 3
False
TRUE [0]


In [ ]:
branch_a = Program().inst(('id', 0)).inst(H(0), MEASURE(0), WAIT, RESET, HALT, H(0), NOP).inst(('TRUE', [0]))
branch_b = Program().defgate('HAD', np.array([[1, 1], [1, -1]]) / np.sqrt(2.)).inst(('HAD', 2))
p.if_then(0, branch_a, branch_b)

## Regular gates

In [8]:
p = Program().inst(('H', 2))
print p
defined_gates = p.defined_gates

H 2



In [9]:
gate_obj = p.actions[0][1]
print gate_obj.operator_name
print gate_obj.parameters
print gate_obj.arguments

H
[]
[2]


In [15]:
type(gate_obj.arguments)

list

In [11]:
np.set_printoptions(precision=4)
exp = tensor_gates(gate_obj, gate_matrix, defined_gates, 3).toarray().astype(np.float64)
print "unitary_gen computed result"
print exp

theory = np.kron(gate_matrix['H'], np.kron(np.eye(2), np.eye(2)))
print "theoretical result"
print theory

print "do they agree? {}".format(np.allclose(exp, theory))

unitary_gen computed result
[[ 0.7071  0.      0.      0.      0.7071  0.      0.      0.    ]
 [ 0.      0.7071  0.      0.      0.      0.7071  0.      0.    ]
 [ 0.      0.      0.7071  0.      0.      0.      0.7071  0.    ]
 [ 0.      0.      0.      0.7071  0.      0.      0.      0.7071]
 [ 0.7071  0.      0.      0.     -0.7071  0.      0.      0.    ]
 [ 0.      0.7071  0.      0.      0.     -0.7071  0.      0.    ]
 [ 0.      0.      0.7071  0.      0.      0.     -0.7071  0.    ]
 [ 0.      0.      0.      0.7071  0.      0.      0.     -0.7071]]
theoretical result
[[ 0.7071  0.      0.      0.      0.7071  0.      0.      0.    ]
 [ 0.      0.7071  0.      0.      0.      0.7071  0.      0.    ]
 [ 0.      0.      0.7071  0.      0.      0.      0.7071  0.    ]
 [ 0.      0.      0.      0.7071  0.      0.      0.      0.7071]
 [ 0.7071  0.      0.      0.     -0.7071 -0.     -0.     -0.    ]
 [ 0.      0.7071  0.      0.     -0.     -0.7071 -0.     -0.    ]
 [ 0.      0. 

/home/richard-zhu/.local/lib/python2.7/site-packages/ipykernel_launcher.py:2: ComplexWarning: Casting complex values to real discards the imaginary part
  


In [12]:
np.kron(np.eye(2), np.eye(2))

array([[ 1.,  0.,  0.,  0.],
       [ 0.,  1.,  0.,  0.],
       [ 0.,  0.,  1.,  0.],
       [ 0.,  0.,  0.,  1.]])

## Defined gates (static and parametric)

In [3]:
p = Program().defgate('HAD', np.array([[0, 1], [1, 0]]))
p.inst(('HAD', 3))
print p

DEFGATE HAD:
    0, 1
    1, 0

HAD 3



In [4]:
dg_obj = p.actions[0][1]
    
print dg_obj.operator_name
print dg_obj.parameters
print dg_obj.arguments

HAD
[]
[3]


In [5]:
defined_gates = {}
defined_gates[p.defined_gates[0].name] = p.defined_gates[0].matrix
print defined_gates

{'HAD': array([[0, 1],
       [1, 0]])}


In [6]:
print tensor_gates(dg_obj, defined_gates, 4).toarray().astype(np.float64)

[[ 0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.]
 [ 1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.  0.  0. 

/home/richard-zhu/.local/lib/python2.7/site-packages/ipykernel_launcher.py:1: ComplexWarning: Casting complex values to real discards the imaginary part
  """Entry point for launching an IPython kernel.


## Measurement

In [19]:
p = Program().inst(H(0))
p.measure(0, [0])

In [20]:
measure_obj = p.actions[1][1]
print p
print measure_obj.operator_name
print measure_obj.parameters
print measure_obj.arguments

H 0
MEASURE 0 [0]

MEASURE
None
(<DirectQubit 0>, <Addr 0>)


In [21]:
print value(measure_obj.arguments[0])
print value(measure_obj.arguments[1])

0
0


In [27]:
p = Program().inst(H(0))
p.measure(0, [0])
p_obj = p.synthesize()

In [33]:
print p_obj[1].arguments

(<DirectQubit 0>, <Addr 0>)


### Measurement tests

In [2]:
from qvm import measurement

In [3]:
nq = 4
wf = np.zeros(2 ** nq).reshape((-1, 1))
wf[0] = 1 / np.sqrt(2)
wf[-1] = 1 / np.sqrt(2)
print wf

[[ 0.70710678]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.70710678]]


In [9]:
measurement(wf, 0)

(1, <16x16 sparse matrix of type '<type 'numpy.complex128'>'
 	with 8 stored elements in Compressed Sparse Row format>, array([[ 0.+0.j],
        [ 0.+0.j],
        [ 0.+0.j],
        [ 0.+0.j],
        [ 0.+0.j],
        [ 0.+0.j],
        [ 0.+0.j],
        [ 0.+0.j],
        [ 0.+0.j],
        [ 0.+0.j],
        [ 0.+0.j],
        [ 0.+0.j],
        [ 0.+0.j],
        [ 0.+0.j],
        [ 0.+0.j],
        [ 1.+0.j]]))